In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# traindata = pd.read_csv("D:\\Homework\\SU\\IST707\\final\\trainSet.csv")
# testdata = pd.read_csv("D:\\Homework\\SU\\IST707\\final\\testSet.csv")
# trainSet = pd.DataFrame(traindata)
# testSet = pd.DataFrame(testdata)
traindata = pd.read_csv("D:\\Homework\\SU\\IST707\\final\\Train_18507.csv")
trainSet = pd.DataFrame(traindata)

# delete the first column
trainSet = trainSet.drop(trainSet.columns[0], axis=1)
print(len(trainSet))
# If a row has missing value, drop it
trainSet = trainSet.dropna(axis=0, how='any')
print(len(trainSet))


18506
17424


In [2]:
# 选出要pivot的列
category_col = ["country","age_group","travel_with","purpose","main_activity","info_source","first_trip_tz"]
print(category_col)
# 循环遍历category_col，将他们都pivot
for col in category_col:
    # pandas内置函数get_dummies，把某一列的所有种类变成列
    one_hot = pd.get_dummies(trainSet[col])
    # 把每一个“种类列”加到dataframe最后，axis=1代表以列的形式加，axis=0是以行的形式
    trainSet = pd.concat([trainSet, one_hot], axis=1)
    # 把原来的pivot之前的列删掉
    trainSet = trainSet.drop(col, axis=1)

['country', 'age_group', 'travel_with', 'purpose', 'main_activity', 'info_source', 'first_trip_tz']


In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
# encode categories into numbers
trainSet[trainSet.columns[0:]] = OrdinalEncoder().fit_transform(trainSet[trainSet.columns[0:]])

# the distribution of the target variable cost_category
trainSet['cost_category'].value_counts()
# merge the cost_category from 0-1 to 0, 2-4 to 1

trainSet['cost_category'] = trainSet['cost_category'].replace([0,1,2], 2)
trainSet['cost_category'] = trainSet['cost_category'].replace([3,4], 0)
trainSet['cost_category'] = trainSet['cost_category'].replace([5], 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainSet.drop('cost_category',axis=1),
                                                    trainSet['cost_category'], test_size=0.20,
                                                    random_state=101)

# Print the distribution of y_test
print(y_train.value_counts())

2.0    7014
1.0    4058
0.0    2867
Name: cost_category, dtype: int64


In [4]:
# 把dataframe写成文件，输出到制定路径
trainSet.to_csv("D:\\Homework\\SU\\IST707\\final\\PCA.csv", index=False, sep=',')

In [63]:
# Decision Tree
from sklearn import tree

clf_dt = Pipeline([
    ('dt', tree.DecisionTreeClassifier())
])

parameters = {
    'dt__max_depth':[2,5,10,15],
    'dt__min_samples_leaf':[2,3,4,5,6,7,8,9,10]
    }

clf_dt = GridSearchCV(clf_dt, parameters, cv = 5)
clf_dt.fit(X_train, y_train)
print("Best Score:", clf_dt.best_score_)
print("Best Params: ", clf_dt.best_params_)
print("Accuracy: ", clf_dt.score(X_test, y_test))

Best Score: 0.4986665214866434
Best Params:  {'dt__max_depth': 2, 'dt__min_samples_leaf': 4}
Accuracy:  0.48948491313623893


In [ ]:
# Naive Bayes
from sklearn import naive_bayes

# Try different naive_bayes algorithm
clf_nb = Pipeline([
    #('standardscaler', StandardScaler()),
    ('nb', naive_bayes.BernoulliNB())
])

parameters = {
    'nb__alpha':[0.5,1],
    'nb__force_alpha':[True, False]
}

clf_nb = GridSearchCV(clf_nb, parameters, cv = 5)
clf_nb.fit(X_train, y_train)
print("Best Score:", clf_nb.best_score_)
print("Best Params: ", clf_nb.best_params_)
print("Accuracy: ", clf_nb.score(X_test, y_test))

In [65]:
# kNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# maxAccuracy = 0
# def train(k):
#     clf = KNeighborsClassifier(n_neighbors = k)
#     clf.fit(X_train, y_train)
#     pred = clf.predict(X_test)
#     return accuracy_score(y_test, pred)


# for i in range(1,11):
#     curAccuracy = train(i)
#     if(curAccuracy > maxAccuracy):
#         maxAccuracy = curAccuracy
#         idx = i
#     print("k =",i,"Accuracy = ",train(i))
# print("Best Accuracy: ",maxAccuracy," k =",idx)

# Add parameter tuning to the model
param_grid = {'n_neighbors': np.arange(1, 11, 1), 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']} 
# n_neighbors: number of neighbors to use
# weights: weight function used in prediction
# metric: the distance metric to use for the tree

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, cv=5)
grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)
print(grid.best_estimator_)

# Print the accuracy of the best model
grid_predictions = grid.predict(X_test)
print(accuracy_score(y_test, grid_predictions))

{'metric': 'manhattan', 'n_neighbors': 10, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=10, weights='distance')
0.40719292898506554


In [ ]:
# SVM
# SVM with Linear Kernel
from sklearn import svm 
param_grid = {'C': [14,12], 'gamma': [0.001, 0.002], 'kernel': ['rbf']} 
# I also tested other parameters, but the accuracy is not better than the above parameters.  'poly', 'sigmoid' are not good.
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, cv=2)
grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)
print(grid.best_estimator_)
# Print the accuracy of the best model
grid_predictions = grid.predict(X_test)
print(accuracy_score(y_test, grid_predictions))

In [ ]:
# SVM with Polynomial Kernel
from sklearn import svm 
print("Without tuning parameters:")
clf_svm_poly = Pipeline([
    ('standardscaler', StandardScaler()),
    ('svc', svm.SVC(kernel='poly'))
])
# train_col = trainSet.columns[0:-1]
# clf_svm_linear.fit(trainSet[train_col], trainSet[trainSet.columns[0]])
# pred_svm_linear = clf_svm_linear.predict(testSet[testSet.columns[1:-1]])
# pred = clf_svm_linear.predict(testSet[testSet.columns[1:-1]])
# print("Accuracy: ",accuracy_score(testSet['cost_category'], pred))


print("Add tuning parameters:")
parameters = {
    'svc__C':np.arange(0.001,1.0, 0.05),
    'svc__degree':[1,2,3],
    'svc__gamma':[0.001, 0.01, 0.1, 0.5],
    'svc__kernel':['poly',]
    }
clf_svm_poly = GridSearchCV(clf_svm_poly, parameters, cv = 5)
clf_svm_poly.fit(X_train, y_train)
print("Best Score:", clf_svm_poly.best_score_)
print("Best Params: ", clf_svm_poly.best_params_)
print("Accuracy: ", clf_svm_poly.score(X_test, y_test))

In [ ]:
# SVM with Radial Kernel
from sklearn import svm 
print("Without tuning parameters:")
clf_svm_rad = Pipeline([
    ('standardscaler', StandardScaler()),
    ('svc', svm.SVC(kernel='rbf'))
])
parameters = {
    'svc__C':[5,10,12],
    'svc__gamma':[0.001, 0.002],
    'svc__kernel':['rbf',]
    }
clf_svm_rad = GridSearchCV(clf_svm_rad, parameters, cv = 5)
clf_svm_rad.fit(X_train, y_train)
print("Best Score:", clf_svm_rad.best_score_)
print("Best Params: ", clf_svm_rad.best_params_)
print("Accuracy: ", clf_svm_rad.score(X_test, y_test))

# param_grid = {'C': [14,12], 'gamma': [0.001, 0.002], 'kernel': ['rbf']} 
# # I also tested other parameters, but the accuracy is not better than the above parameters.  'poly', 'sigmoid' are not good.
# grid = GridSearchCV(svm.SVC(), param_grid, refit = True, cv=2)
# grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)
print(grid.best_estimator_)
# Print the accuracy of the best model
grid_predictions = grid.predict(X_test)
print(accuracy_score(y_test, grid_predictions))

In [ ]:
# K-means Cluster
from sklearn.cluster import KMeans

clf_km = Pipeline([
    ('standardScaler', StandardScaler()),
    ('km', KMeans())
])
parameters = {
    'km__n_clusters':range(3, 8, 1),
    #'km__algorithm':['lloyd', 'elkan']
}
clf_km = GridSearchCV(clf_km, parameters)
clf_km.fit(X_train)
print("Accuracy: ", clf_km.score(X_test))

In [66]:
from sklearn.ensemble import RandomForestClassifier
param_grid = {'n_estimators': range(290,350,10), 'max_depth': range(10,15,1), 'criterion': ['gini']} 
# I have tested a lot of parameters, the n_estimators value from 1 to 300, the max_depth value from 1 to 20, the criterion value from 'gini' to 'entropy'.
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, cv=2)
grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)
print(grid.best_estimator_)

# Print the accuracy of the best model
grid_predictions = grid.predict(X_test)
print(accuracy_score(y_test, grid_predictions))

{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 340}
RandomForestClassifier(max_depth=10, n_estimators=340)
0.48948491313623893
